In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from timm import create_model

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define dataset path (Ensure dataset has folders: aca_bd, aca_md, aca_pd)
dataset_path = "/content/drive/MyDrive/Final"

In [4]:
# Data Transformations (Augmentation for better performance)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# Load Dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=data_transforms)


In [6]:
# Split dataset into Train (80%), Validation (10%), Test (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


In [7]:
# Create Data Loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# Load Pretrained Vision Transformer Model (ViT)
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=3)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [9]:
# Add Dropout to Fully Connected Layers
for name, module in model.named_children():
    if isinstance(module, nn.Linear):
        setattr(model, name, nn.Sequential(
            module,
            nn.Dropout(0.3)  # 30% dropout to prevent overfitting
        ))

In [10]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
# Training Function
def train_model(model, train_loader, val_loader, epochs=30):
    best_acc = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
        scheduler.step(val_acc)  # Adjust learning rate based on validation accuracy
         # Save Best Model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "differentiation_model.pth")


In [12]:
# Evaluation Function
def evaluate_model(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

In [13]:
# Train the Model
train_model(model, train_loader, val_loader, epochs=30)

Epoch 1/30: Train Loss: 1.6025, Train Acc: 35.27%, Val Acc: 39.29%
Epoch 2/30: Train Loss: 1.1980, Train Acc: 37.50%, Val Acc: 28.57%
Epoch 3/30: Train Loss: 1.0717, Train Acc: 40.62%, Val Acc: 50.00%
Epoch 4/30: Train Loss: 1.0064, Train Acc: 46.88%, Val Acc: 60.71%
Epoch 5/30: Train Loss: 0.9888, Train Acc: 50.89%, Val Acc: 64.29%
Epoch 6/30: Train Loss: 0.9503, Train Acc: 52.68%, Val Acc: 64.29%
Epoch 7/30: Train Loss: 0.8793, Train Acc: 60.71%, Val Acc: 71.43%
Epoch 8/30: Train Loss: 0.8369, Train Acc: 59.82%, Val Acc: 75.00%
Epoch 9/30: Train Loss: 0.7590, Train Acc: 66.52%, Val Acc: 78.57%
Epoch 10/30: Train Loss: 0.7106, Train Acc: 65.18%, Val Acc: 67.86%
Epoch 11/30: Train Loss: 0.7358, Train Acc: 62.95%, Val Acc: 60.71%
Epoch 12/30: Train Loss: 0.7085, Train Acc: 68.30%, Val Acc: 75.00%
Epoch 13/30: Train Loss: 0.6177, Train Acc: 70.09%, Val Acc: 85.71%
Epoch 14/30: Train Loss: 0.5673, Train Acc: 77.23%, Val Acc: 85.71%
Epoch 15/30: Train Loss: 0.5022, Train Acc: 79.02%, Val A

In [14]:
# Test the Model
test_acc = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 89.29%


In [21]:
def predict_image(image_path, model, device):
    from PIL import Image
    import torchvision.transforms as transforms

    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    return predicted.item()

# Example usage
predicted_class = predict_image("/content/test_image.jpeg", model, device)
print(f"Predicted Class: {predicted_class}")
#Predicted 0: aca_bd= well differentiated
#Predicted 1: aca_md= moderately differentiated
#Predicted 2: aca_pd= poorly differentiated


Predicted Class: 1


In [22]:
torch.save(model.state_dict(), "/content/drive/MyDrive/final_vit_model.pth")